# Identificação de Homólogos

In [9]:
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from scripts_python import BLASTer

a = %pwd
wd = (a.rsplit('/',2))[0]
seq_id = "NM_001114380"
prot_seq_id = "P20701"
gene = "ITGAL"
file_blast_ncbi = "00D8F7PY016-Alignment.xml"
file_blast_uniprot = "B20210131A94466D2655679D1FD8953E075198DA8030E49B.fasta"

In [10]:
result_handle= open(os.path.join( wd,"data/homologue", gene, file_blast_ncbi))
blast_ncbi = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas {0} seq.'.format(len(blast_ncbi.alignments)))

Foram carregadas 100 seq.


## Resultados do BLAST no NCBI

In [11]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import re

query_len = blast_ncbi.query_length
homologue = []
sources = {}

print("Accession", "Identity%")
for aln in blast_ncbi.alignments:
    for HSP in aln.hsps:
        if HSP.score > 80 and HSP.expect < 1.0e-10 and HSP.identities/HSP.align_length > 0.80:
            print(aln.accession,HSP.identities/HSP.align_length)
            species = re.findall('\[[^\]]*\]', aln.hit_def)[0].replace("[","").replace("]","")
            if species not in sources:
                sources[species] = 1
            else:
                sources[species] +=1
            homologue.append(SeqRecord(Seq(HSP.sbjct), aln.title.split( ">", 1)[0], "", ""))


print("Total:",len(homologue))

Accession Identity%
NP_002200 1.0
BAG36913 0.9991452991452991
XP_005255370 0.9991452991452991
CAA68747 0.9982905982905983
NP_001029341 0.9923076923076923
XP_034795955 0.9923076923076923
PNI12205 0.9923076923076923
XP_034795956 0.9914529914529915
XP_024209187 0.9914529914529915
EAW52244 0.9566639411283728
AAC31672 0.955846279640229
XP_024089734 0.9777777777777777
XP_032011574 0.9709401709401709
XP_024209188 0.976068376068376
XP_030653892 0.9675213675213675
XP_024209189 0.9675213675213675
XP_023046865 0.958974358974359
XP_023046867 0.958904109589041
XP_023046866 0.9581196581196582
XP_010386254 0.9555555555555556
EHH31590 0.9538461538461539
XP_025225979 0.9555555555555556
XP_037845568 0.9573742540494459
XP_009197106 0.9564102564102565
XP_033074013 0.9512820512820512
XP_014981430 0.9529914529914529
EHH60316 0.9511986301369864
XP_033074015 0.9511986301369864
XP_033074014 0.9504273504273504
XP_017736913 0.9512820512820512
XP_028697390 0.9253112033195021
XP_011544151 1.0
XP_034795957 0.991620

In [12]:
for source in sources.keys():
    print(source, ":", sources[source])

SeqIO.write(homologue, os.path.join(wd,"data/homologue", gene, f"{seq_id}_{gene}.fasta"), "fasta")

Homo sapiens : 11
Pan troglodytes : 11
Pan paniscus : 3
Pongo abelii : 3
Hylobates moloch : 1
Nomascus leucogenys : 3
Piliocolobus tephrosceles : 4
Rhinopithecus roxellana : 1
Macaca mulatta : 3
Theropithecus gelada : 1
Chlorocebus sabaeus : 1
Papio anubis : 4
Trachypithecus francoisi : 4
Macaca fascicularis : 1
Rhinopithecus bieti : 1
Mandrillus leucophaeus : 1
Sapajus apella : 2
Callithrix jacchus : 1
Aotus nancymaae : 1
Saimiri boliviensis boliviensis : 1
Macaca nemestrina : 1
synthetic construct : 2
Galeopterus variegatus : 1
Equus asinus : 2
Equus caballus : 2
Equus przewalskii : 1
Carlito syrichta : 1
Propithecus coquereli : 1
Microcebus murinus : 4
Zalophus californianus : 2
Callorhinus ursinus : 4
Odobenus rosmarus divergens : 1
Eumetopias jubatus : 2
Balaenoptera musculus : 2
Pteropus alecto : 2
Sousa chinensis : 1
Lagenorhynchus obliquidens : 2
Tursiops truncatus : 2
Pteropus vampyrus : 2
Globicephala melas : 2
Phoca vitulina : 2
Neophocaena asiaeorientalis asiaeorientalis : 

105

### Resultados UNIPROT

#### Comentários

In [13]:
path = os.path.join(wd, "data/homologue", gene, file_blast_uniprot)
blast_uniprot_raw = SeqIO.parse(path, format="fasta")
blast_uniprot = []
blast_uniprot_id = []

for protein in blast_uniprot_raw:
    blast_uniprot.append(protein)
    seq_id = re.findall('\|[^|]*\|', protein.id)[0].replace("|","")
    blast_uniprot_id.append(seq_id)

In [14]:
from wget import download

for accession in blast_uniprot_id:
    url = "https://www.uniprot.org/uniprot/{0}.xml".format(accession)
    path = os.path.join(wd, "data/homologue", gene, "uniprot","{0}.xml".format(accession))
    download(url, path)


In [15]:
import re

blast_matches = []

for accession in blast_uniprot_id:
    path = os.path.join(wd, "data/homologue", gene,"uniprot","{0}.xml".format(accession))
    blast_matches.append(SeqIO.read(path, format="uniprot-xml"))

comments = {}
an_domains = {}
keywords = {}
for match in blast_matches:
    if "keywords" in match.annotations.keys():
        for keyword in match.annotations["keywords"]:
            if keyword in keywords:
                keywords[keyword] = ", ".join([keywords[keyword],match.id])
            else:
                keywords[keyword] = match.id
    for annotation in match.annotations.keys():
        if re.match("(comment_)[a-z]*", annotation):
            comment_type = annotation.split("_",1)[1]
            if comment_type not in comments.keys():
                comments[comment_type] = {}
            comment = match.annotations[annotation][0]
            if comment in comments[comment_type].keys():
                comments[comment_type][comment] = ", ".join([comments[comment_type][comment], match.id])
            else:
                 comments[comment_type][comment] = match.id

for comment_type in comments.keys():
    print(comment_type)
    for comment in comments[comment_type].keys():
        print(comment, ":", comments[comment_type][comment])
    print("\n")

print("Keywords")
for keyword in keywords.keys():
    print(keyword, ":", keywords[keyword])
print("\n")

function
Integrin ITGAL/ITGB2 is a receptor for ICAM1, ICAM2, ICAM3 and ICAM4. Integrin ITGAL/ITGB2 is a receptor for F11R (PubMed:11812992, PubMed:15528364). Integin ITGAL/ITGB2 is a receptor for the secreted form of ubiquitin-like protein ISG15; the interaction is mediated by ITGAL (PubMed:29100055). Involved in a variety of immune phenomena including leukocyte-endothelial cell interaction, cytotoxic T-cell mediated killing, and antibody dependent killing by granulocytes and monocytes. Contributes to natural killer cell cytotoxicity (PubMed:15356110). Involved in leukocyte adhesion and transmigration of leukocytes including T-cells and neutrophils (PubMed:11812992). Required for generation of common lymphoid progenitor cells in bone marrow, indicating a role in lymphopoiesis (By similarity). Integrin ITGAL/ITGB2 in association with ICAM3, contributes to apoptotic neutrophil phagocytosis by macrophages (PubMed:23775590). : P20701
Integrin ITGAL/ITGB2 is a receptor for ICAM1, ICAM2, IC

As anotações confirmam a relação das proteínas homólogas e o suporte ao sistema imunitário, indicando que esta função é conservada entre as espécies de mamíferos, portadores deste gene.

### Features

In [16]:
features = {}
for match in blast_matches:
    for feature in match.features:
        if re.search("(domain|motif|bond|region)", feature.type):
            feature_type = feature.type
            if feature_type not in features.keys():
                features[feature_type] = {}

            feature_desc = str(feature.location)
            if "description" in feature.qualifiers.keys():
                feature_desc = " ".join([feature_desc,feature.qualifiers["description"]])

            if feature_desc in features[feature_type].keys():
                features[feature_type][feature_desc] = ", ".join([features[feature_type][feature_desc], match.id])
            else:
                 features[feature_type][feature_desc] = match.id

for feature_type in features.keys():
    print(feature_type)
    for feature in features[feature_type].keys():
        print(feature, ":", features[feature_type][feature])
    print("\n")

topological domain
[25:1090] Extracellular : P20701
[1111:1170] Cytoplasmic : P20701
[23:1084] Extracellular : P61625
[1105:1165] Cytoplasmic : P61625
[23:1087] Extracellular : P24063
[1108:1163] Cytoplasmic : P24063
[16:1104] Extracellular : P11215
[1128:1152] Cytoplasmic : P11215
[16:1105] Extracellular : P05555
[1129:1153] Cytoplasmic : P05555
[19:1116] Extracellular : Q9QXH4
[1137:1169] Cytoplasmic : Q9QXH4
[19:1107] Extracellular : P20702
[1128:1163] Cytoplasmic : P20702
[19:1100] Extracellular : Q9QYE7
[1121:1161] Cytoplasmic : Q9QYE7
[26:1107] Extracellular : Q3V0T4
[1128:1168] Cytoplasmic : Q3V0T4
[17:1099] Extracellular : Q13349
[1120:1161] Cytoplasmic : Q13349
[22:1141] Extracellular : Q9UKX5, P61622
[1164:1188] Cytoplasmic : Q9UKX5, P61622
[22:1122] Extracellular : O75578
[1145:1167] Cytoplasmic : O75578


transmembrane region
[1090:1111] Helical : P20701
[1084:1105] Helical : P61625
[1087:1108] Helical : P24063
[1104:1128] Helical : P11215
[1105:1129] Helical : P05555
[1116

Apesar da varialibilidade da localização das *features*, expectável dado que estas não se encontram alinhadas, podemos perceber que estas são comuns entre os homólogos, identificando locais de interesse para estudo.